# 2-4 ネットワークモデルの実装

In [13]:
# パッケージのimport
from math import sqrt
from itertools import product

import pandas as pd
import torch
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

# モジュールvggを実装

In [14]:
# 35層にわたる、vggモジュールを作成
def make_vgg():
    layers =[]
    in_channels =3  #色チャネル数
    
    # vggモジュールで使用する畳み込み層やマックスプーリングのチャネル
    cfg = [64 , 64 , 'M' , 128 ,128 ,'M' , 256, 256 ,256, 'MC' , 512 ,512 , 512  , 'M' , 512 ,512 ,512]
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2 , stride=2)]
        elif v == 'MC':
            # ceilは出力サイズを、計算結果（float）に対して、切り上げで整数にするモード
            # デフォルトでは出力サイズを計算結果（float）に対して、切り下げで整数にするfloorモード
            layers += [nn.MaxPool2d(kernel_size=2 ,stride=2 , ceil_mode=True)]
        else:
            conv2d =nn.Conv2d(in_channels , v , kernel_size=3 , padding=1)
            layers  += [conv2d  , nn.ReLU(inplace=True)]
            in_channels = v
            
    pool5 =nn.MaxPool2d(kernel_size=3 ,stride=1 ,  padding=1)
    conv6 = nn.Conv2d(512 , 1024 , kernel_size=3 , padding=6 ,  dilation=6)
    conv7 = nn.Conv2d(1024 ,1024 , kernel_size=1)
    layers += [pool5 , conv6 , nn.ReLU(inplace=True) , conv7 ,nn.ReLU(inplace=True)]
    
    return nn.ModuleList(layers)
    
#動作確認
vgg_test = make_vgg()
print(vgg_test)
       
    
            

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz